# Create a plot of Lab Analysis Results for all months this year

In [ ]:
# This program helps you generate the dataframe for each month per area of sampling

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dateutil.parser import parse
from itertools import compress
from collections import OrderedDict
from datetime import date
import datetime
import re
from datetime import timedelta
import openpyxl
from sklearn.metrics import mean_squared_error
import plotly.express as px

import plotly.graph_objects as go
from plotly.subplots import make_subplots

#---------------------
#
def time_cleaner(x):
    time_regex = re.compile(r'[0-8a-zA-Z:]', re.IGNORECASE| re.VERBOSE|re.DOTALL)
    timeact = time_regex.findall(x)
    newtime = ''.join(timeact)
    t = parse(x)
    new_str_time = str(t.hour)+':'+str(t.minute)+':'+str(t.second)
    return new_str_time
#
#-------------------
#
def ave_filler(dfname, colname):
    dfcn = dfname[colname]
    try:        
        dfcn = pd.to_numeric(dfcn, errors='coerce')
    except:
        print('error found in' + str(colname))
        pass
    try:
        dfname[colname] = dfcn.where(dfcn.notnull(), other=(dfcn.fillna(method='ffill') + dfcn.fillna(method='bfill'))/2)
    except:
        print('nan not fixed' + str(colname))   
    return dfname

#----------------------
#
#generate_df_from_analysis: this function creates dataframe by getting laboratory results by taking the excel values from
#    every row in a specified analysis column. An example of analysis column is the column for Zn AAS analysis for NTRL.
#    The critical part of this code is in making sure that the dates and time from the laboratory analysis excel file are
#    parsed and converted into the correct datetime format.
def generate_df_from_analysis(workbookTab, col_sampleDate, col_sampleTime, col_analysis, col_title):
    #get the start and end sample dates
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
    #gets the date day from the B column of the analysis file if there is value in the cell
    for i in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
            Smpl_day_index.append(i)
    #get the ending row that has date day
    for j in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
            smpl_end_index = j
    for y in range(0, len(Smpl_day_index)):
        if Smpl_day_index[y] != Smpl_day_index[-1]:      
            for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                if workbookTab[str(col_analysis)+str(x)].value is None: 
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

        if Smpl_day_index[y] == Smpl_day_index[-1]:
            for x in range(Smpl_day_index[y], smpl_end_index+1):
                if workbookTab[str(col_analysis)+str(x)].value is None:   
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

    Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
    Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
    Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
    Chem_Element_df.set_index('DATETIME', inplace=True)

    return Chem_Element_df
#-----------------------------------------------------End----------------------------------------------------------------
#
#
#------------------------------------------------------------------------------------------------------------------------
#
def generate_add_trace(figname, df_elemental, axis_bool):
    colname = df_elemental.columns[0]
    figname.add_trace(go.Scatter(x=df_elemental.index, y=df_elemental[colname], name=colname), secondary_y=axis_bool,)
#
#----------------------------------------------------End-----------------------------------------------------------------
#
#
class LabResults():
    def __init__(self, monthyear, filePath):
        #____monthyear format examle is datetime.date(2020,7,1)
        #____filepath example is '\\thgobna001\userdata\THPAL\...\ANALYSIS RESULTS 2020\7) July _2020.xlsx'
        #____add r' at the start of the filepath to apply the regex that allows the use of the filepath as raw
        #____sef.workbook is the uploaded excel workbook of the entire analysis files so it takes time to load
        #____Create an instance of the workbook by giving it a varible name with the sample format: Jan_2020
        
        import openpyxl
        import io
        with open(filePath, "rb") as f:
            in_mem_file = io.BytesIO(f.read())
        
        self.workbook = openpyxl.load_workbook(in_mem_file, data_only=True)
        self.monthyear = monthyear   
        
    def generate_df_from_analysis(self, tabname, list_lab_index):
        #____tabname is a string type name of the tab in the laboratory analysis excel file.
        #____-------Example tabnames: '104PU01', '105TK03', '106TK01'
        #____-------Inside these tabs are the specific analysis results done on the sample taken from the 'tabname' area .
        #____list_lab_index is a list with the sample format is: ['B', 'C', 'E', '104_Pb']
        #____-------the first letter 'B' is the column letter where the day of the month is written as integers 1,2,3..30
        #____-------the second letter 'C' is the column letter where the sampling time is written; e.g. '7:00:00 AM'
        #____-------the third letter 'E' is the column where the results of the specific analysis are listed
        #____-------the last item in the list is the string you'll use as the new title; e.g. '104_Pb'
        
        workbookTab = self.workbook[tabname]
        self.list_lab_index = list_lab_index
        col_sampleDate = self.list_lab_index[0]
        col_sampleTime = self.list_lab_index[1]
        col_analysis = self.list_lab_index[2]
        col_title = self.list_lab_index[3]
        currentmonthyr = self.monthyear
        
        #get the start and end sample dates
        Smpl_day_index = list()
        Chem_Element_datelist = list()
        Chem_Element_list = list()
        Smpl_day_index = list()
        Chem_Element_datelist = list()
        Chem_Element_list = list()
        plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
        #gets the date day from the B column of the analysis file if there is value in the cell
        for i in range(1, workbookTab.max_row):
            if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
                Smpl_day_index.append(i)
        #get the ending row that has date day
        for j in range(1, workbookTab.max_row):
            if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
                smpl_end_index = j
        for y in range(0, len(Smpl_day_index)):
            if Smpl_day_index[y] != Smpl_day_index[-1]:      
                for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                    if workbookTab[str(col_analysis)+str(x)].value is None: 
                        pass
                    else:
                        if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                            date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                            Chem_Element_datelist.append(parse(date))  
                            chem_element = workbookTab[str(col_analysis)+str(x)].value
                            Chem_Element_list.append(chem_element)
                        else:
                            date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                            Chem_Element_datelist.append(parse(date))  
                            chem_element = workbookTab[str(col_analysis)+str(x)].value
                            Chem_Element_list.append(chem_element)

            if Smpl_day_index[y] == Smpl_day_index[-1]:
                for x in range(Smpl_day_index[y], smpl_end_index+1):
                    if workbookTab[str(col_analysis)+str(x)].value is None:   
                        pass
                    else:
                        if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                            date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                            Chem_Element_datelist.append(parse(date))
                            chem_element = workbookTab[str(col_analysis)+str(x)].value
                            Chem_Element_list.append(chem_element)
                        else:
                            date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                            Chem_Element_datelist.append(parse(date))
                            chem_element = workbookTab[str(col_analysis)+str(x)].value
                            Chem_Element_list.append(chem_element)

        Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
        Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
        Chem_Element_df.columns = [col_title, 'DATETIME']#------------------------------------------------check if list title is updated
        Chem_Element_df.set_index('DATETIME', inplace=True)

        return Chem_Element_df
#---------------------------------------------------------------End---------------------------------------------------------------------
#
#
def join_DF(dfsList1):
    df = dfsList1[0]
    for i in range(1,len(dfsList1)):
        df = df.join(dfsList1[i], how='outer')
    return df
#
#---------------------------------------------------------------End---------------------------------------------------------------------
#
#
def append_monthsDF(dfsListmonthly): #takes the list of dataframes from monthly lab results of one sampling area
    df = dfsListmonthly[0]
    for i in range(1,len(dfsListmonthly)):
        df = df.append(dfsListmonthly[i])
    try:
        df.set_index('DATETIME', inplace=True)
    except:
        pass
    return df  
#
#
#---------------------------------------------------------------End----------------------------------------------------------------------
#
#
def generate_combined_df_by_month(month_WB, sample_area, analysis_type):
    df_List = []
    for i in analysis_type:
        df_analysis = month_WB.generate_df_from_analysis(sample_area, i)
        df_List.append(df_analysis)
    for i in df_List:
        df = join_DF(df_List)
    return df

#
#
#---------------------------------------------------------------End----------------------------------------------------------------------
#
#  make_df_for_each_FT01_by_month_then_export_excel: This function exports the 
def make_df_for_each_FT01_by_month_then_export_excel(monthNumber, analysis_type_FT01AtoF_list):
    month_ = LabResults(currentmonthyr_list[monthNumber-1], filepath_list[monthNumber-1])
    letters = 'ABCDEF'
    FT01_months_df = []
    for i in range(0,6):
        FT01_ = generate_combined_df_by_month(month_, '105FT01', analysis_type_FT01AtoF_list[i])
        FT01_.to_excel(r'C:\Users\v.t.flores\Documents\FT01{}_2020_0{}.xlsx'.format(letters[i], monthNumber))
#
#
#---------------------------------------------------------------End----------------------------------------------------------------------
#
# generate_add_trace: this function is for creating plots using plotly express
def generate_add_trace(figname, df_elemental, axis_bool):
    colname = df_elemental.columns[0]
    figname.add_trace(go.Scatter(x=df_elemental.index, y=df_elemental[colname], name=colname), secondary_y=axis_bool,)   
#
#
#--------------------------------------------------------------End-----------------------------------------------------------------------
#
#
def generate_plot_from_df(figname, df,df_axis_truefalse_list):
    for i in range(0, len(df.columns)):
        colname = df.columns[i]
        figname.add_trace(go.Scatter(x=df[colname].index, y=df[colname], name=colname), secondary_y=df_axis_truefalse_list[i],)
#
#
#--------------------------------------------------------------End-----------------------------------------------------------------------

def make_a_graph(Samplesource_name, figname, df, axisTrueFalse):
    generate_plot_from_df(figname, df, axisTrueFalse)
    print(Samplesource_name)
    print(df.columns)

    t = datetime.datetime.now()
    title_plot = Samplesource_name+"<b>Laboratory Analysis Results</b>" + '<br>' + 'Updated as of:' + str(t.strftime("%m/%d/%Y %H:%M:%S"))+'H'
    
    figname.update_layout(title_text=title_plot)                                 # Add figure title
    
    figname.update_xaxes(title_text="DateTime")                                  # Set x-axis title
    
    figname.update_yaxes(title_text= "<b> </b>", secondary_y=False)              # Set y-axes titles primary
    figname.update_yaxes(title_text="<b> </b>", secondary_y=True)                # Set y-axes titles secondary
    
    #this function needs upgrade to correctly label the axis
    return figname
#
#
#--------------------------------------------------------------End-----------------------------------------------------------------------
#
def replace_0_with_nan(df, dfcol_list): # the argument df is the dataframe, dfcol_list is the list of column names we want to clean up
    for i in dfcol_list:
        df[i] = np.where(df[i] == 0, np.nan, df[i])
#       
#--------------------------------------------------------------End-----------------------------------------------------------------------        
#
#
def convert_to_numeric(df):
    for i in df.columns:
        try:
            df[i] = pd.to_numeric(df[i], errors='coerce')
        except:
            print(i, " has error.")
#            
#--------------------------------------------------------------End-----------------------------------------------------------------------            

In [ ]:
FT01 = pd.read_excel(r'C:\Users\v.t.flores\Documents\FT01AtoF_2020_0107_DF.xlsx')
NTRL = pd.read_excel(r'C:\Users\v.t.flores\Documents\NTRL_Jan_Jul2020.xlsx')
DeZnTK03 = pd.read_excel(r'C:\Users\v.t.flores\Documents\TK03_laboResults_2020_01to07DF.xlsx')
DeZnTK05 = pd.read_excel(r'C:\Users\v.t.flores\Documents\TK05_2020_DF.xlsx')
MSTK01 = pd.read_excel(r'C:\Users\v.t.flores\Documents\MS_TK01_2020_01_07_DF.xlsx')
MSVE04 = pd.read_excel(r'C:\Users\v.t.flores\Documents\MS_VE04_2020_01_07_DF.xlsx')

NTRL.set_index('DATETIME', inplace=True)
NTRL = NTRL.drop('Unnamed: 0', axis=1)
FT01.set_index('DATETIME', inplace=True)
DeZnTK03.set_index('DATETIME', inplace=True)
DeZnTK03 = DeZnTK03.drop('Unnamed: 0', axis=1)
DeZnTK05.set_index('DATETIME', inplace=True)
DeZnTK05 = DeZnTK05.drop(DeZnTK05.columns[0], axis=1)
MSTK01.set_index('DATETIME', inplace=True)
MSVE04.set_index('DATETIME', inplace=True)

In [ ]:
#-------------------------------------This code block contains the lists used in the succeeding functions-------------------------------
DeZnTK03_for_cleaning_list = ['104PU01_Ni',
                              '105TK05_Ni',
                              'Ni_Loss_TK05',
                              '105TK05_Zn',
                              '106TK01_Zn']
Areas_df_list = [FT01,
                 NTRL,
                 DeZnTK03,
                 DeZnTK05,
                 MSTK01,
                 MSVE04]

Areas_name_list = ['FT01',
                 'NTRL',
                 'DeZnTK03',
                 'DeZnTK05',
                 'MSTK01',
                 'MSVE04']

replace_0_with_nan(DeZnTK03, DeZnTK03_for_cleaning_list)
convert_to_numeric(FT01)
convert_to_numeric(NTRL)
convert_to_numeric(DeZnTK03)
convert_to_numeric(DeZnTK05)
convert_to_numeric(MSTK01)
convert_to_numeric(MSVE04)

NTRL = NTRL.interpolate(method='linear', limit_direction='forward', axis=0)
NTRL.dropna(how='any', inplace=True)
DeZnTK03 = DeZnTK03.interpolate(method='linear', limit_direction='forward', axis=0)
DeZnTK03.dropna(how='any', inplace=True)
DeZnTK05 = DeZnTK05.interpolate(method='linear', limit_direction='forward', axis=0)
DeZnTK05.dropna(how='any', inplace=True)
MSTK01 = MSTK01.interpolate(method='linear', limit_direction='forward', axis=0)
MSTK01.dropna(how='any', inplace=True)
MSVE04 = MSVE04.interpolate(method='linear', limit_direction='forward', axis=0)
MSVE04.dropna(how='any', inplace=True)

In [ ]:
#========================================For Polishing Filter Laboratory Results
#
# This code cleans up the polishing filter data. The output is the dataframe AtoF which is the compiled Zn, ntu, Zn_dr of all the filters

A = FT01[FT01.columns[0:3]]
A['Filter'] = 'FT01A'
B = FT01[FT01.columns[3:6]]
B['Filter'] = 'FT01B'
C = FT01[FT01.columns[6:9]]
C['Filter'] = 'FT01C'
D = FT01[FT01.columns[9:12]]
D['Filter'] = 'FT01D'
E = FT01[FT01.columns[12:15]]
E['Filter'] = 'FT01E'
F = FT01[FT01.columns[15:18]]
F['Filter'] = 'FT01F'

A.reset_index(inplace=True)
B.reset_index(inplace=True)
C.reset_index(inplace=True)
D.reset_index(inplace=True)
E.reset_index(inplace=True)
F.reset_index(inplace=True)

A.columns = ['DATETIME', 'Zn', 'ntu', 'Zn_dr', 'Filter']
B.columns = ['DATETIME', 'Zn', 'ntu', 'Zn_dr', 'Filter']
C.columns = ['DATETIME', 'Zn', 'ntu', 'Zn_dr', 'Filter']
D.columns = ['DATETIME', 'Zn', 'ntu', 'Zn_dr', 'Filter']
E.columns = ['DATETIME', 'Zn', 'ntu', 'Zn_dr', 'Filter']
F.columns = ['DATETIME', 'Zn', 'ntu', 'Zn_dr', 'Filter']

AtoF = A.append(B, ignore_index =True).append(C, ignore_index = True).append(D, ignore_index=True).append(E, ignore_index=True).append(F, ignore_index=True)
AtoF = A.append(B, ignore_index =True).append(C, ignore_index = True).append(D, ignore_index=True).append(E, ignore_index=True).append(F, ignore_index=True)
AtoF.dropna(how='any', inplace=True)
AtoF.columns = ['DATETIME', 'Zn', 'ntu', 'Zn_dr', 'Filter']

In [1]:
#--------------------------------------------------------------------- Polishing Filter-----------------------------------------------------------------
#
#
figPolishing = px.scatter(AtoF, x="DATETIME", y="Zn_dr", size="Zn", color="Filter", hover_name="ntu", size_max=40)
title_plotPolishing = "<b>Polishing Filter</b>" + '<br>' + 'Marker Size = Zn conc' + ' ; ' + 'On Hover = NTU'
figPolishing.update_layout(title_text=title_plotPolishing)
#
#-------------------------------------------------------------------------End----------------------------------------------------------------------------

NameError: name 'px' is not defined

In [ ]:
#----------------------------------------------------------------------  NTRL----------------------------------------------------------------------------
#
#
figNTRL = make_subplots(specs=[[{"secondary_y": True}]])
figNTRL_axis_truefalse_list = [False, False, False, True, False,
                               True, False, False, False, True,
                               True, False, True, False, False, 
                               False]
generate_plot_from_df(figNTRL, NTRL, figNTRL_axis_truefalse_list)

#title_plot = 'Zn Drop Trend InterActive \n' + '\n Updated as of: ' + str(datetime.datetime.now())
t = datetime.datetime.now()
title_plotNTRL = "<b>NTRL Laboratory Analysis Results</b>" + '<br>' + 'Updated as of:' + str(t.strftime("%m/%d/%Y %H:%M:%S"))+'H'
# Add figure title
figNTRL.update_layout(title_text=title_plotNTRL)
# Set x-axis title
figNTRL.update_xaxes(title_text="DateTime")
# Set y-axes titles
figNTRL.update_yaxes(title_text="<b>Pb, Zn , Cu, Co, Cr, Ca, Si, Fe2</b>", secondary_y=False)
figNTRL.update_yaxes(title_text="<b>Ni, Mn, Al, Mg, pH, ORP</b>", secondary_y=True)
#
#-------------------------------------------------------------------------End----------------------------------------------------------------------------

In [ ]:
#---------------------------------------------------------------------- 105TK03 -------------------------------------------------------------------------
#
#
figDeZnTK03 = make_subplots(specs=[[{"secondary_y": True}]])
figDeZNTK03_axis_truefalse_list = [True, False, False, True, True,
                                   False, True, False, False, False,
                                   False]
generate_plot_from_df(figDeZnTK03, DeZnTK03, figDeZNTK03_axis_truefalse_list)

#title_plot = 'Zn Drop Trend InterActive \n' + '\n Updated as of: ' + str(datetime.datetime.now())
title_plotDeZnTK03 = "<b>105TK03 Laboratory Analysis Results</b>" + '<br>' + 'Updated as of:' + str(t.strftime("%m/%d/%Y %H:%M:%S"))+'H'
# Add figure title and label
figDeZnTK03.update_layout(title_text=title_plotDeZnTK03)
figDeZnTK03.update_xaxes(title_text="DateTime") # Set x-axis title
figDeZnTK03.update_yaxes(title_text="TK03_Zn, TK05_Zn , 106TK01_Zn, 106TK01_Zn/Ni, VE04_Zn%, 106TH01_Zn%, Ni_Loss", secondary_y=False) # Set y-axes titles primary axis
figDeZnTK03.update_yaxes(title_text="Zn_ChangeRatio, 106TK01_Zn_DropRate, 104PU01_Ni, Mg, 105TK05_Ni, Ni_Loss_TK05", secondary_y=True) # Set y-axes titles secondary axis
#
#-------------------------------------------------------------------------End----------------------------------------------------------------------------

In [ ]:
#---------  105TK05
figDeZnTK05 = make_subplots(specs=[[{"secondary_y": True}]])
figDeZnTK05_axis_truefalse_list = [False, False, False, True, True, 
                            True, False, False, False, True, 
                            True, True, False, True, True]
generate_plot_from_df(figDeZnTK05, DeZnTK05, figDeZnTK05_axis_truefalse_list)

#title_plot = 'Zn Drop Trend InterActive \n' + '\n Updated as of: ' + str(datetime.datetime.now())
title_plotDeZnTK05 = "<b>105TK05 Laboratory Analysis Results</b>" + '<br>' + 'Updated as of:' + str(t.strftime("%m/%d/%Y %H:%M:%S"))+'H'
# Add figure title and label
figDeZnTK05.update_layout(title_text=title_plotDeZnTK05)
figDeZnTK05.update_xaxes(title_text="DateTime") # Set x-axis title
figDeZnTK05.update_yaxes(title_text="<b>Pb, Zn , Cu, Co, Cr, Ca, Si, Fe2</b>", secondary_y=False) # Set y-axes titles primary axis
figDeZnTK05.update_yaxes(title_text="<b>Ni, Mn, Al, Mg, pH, ORP</b>", secondary_y=True) # Set y-axes titles secondary axis

#-------------------------------------------------------------------------End----------------------------------------------------------------------------

In [ ]:
#---------  106TK01

figMSTK01 = make_subplots(specs=[[{"secondary_y": True}]])
figMSTK01_axis_truefalse_list = [False, False, False, True, True, 
                            True, True, True, True, True, 
                            True, True, True, False, True]
generate_plot_from_df(figMSTK01, MSTK01, figMSTK01_axis_truefalse_list)

#title_plot = 'Zn Drop Trend InterActive \n' + '\n Updated as of: ' + str(datetime.datetime.now())
title_plotMSTK01 = "<b>106TK01 Laboratory Analysis Results</b>" + '<br>' + 'Updated as of:' + str(t.strftime("%m/%d/%Y %H:%M:%S"))+'H'
# Add figure title and label
figMSTK01.update_layout(title_text=title_plotMSTK01)
figMSTK01.update_xaxes(title_text="DateTime") # Set x-axis title
figMSTK01.update_yaxes(title_text="Pb, Zn , Cu, Co, Cr, Ca, Si, Fe, ZnNi", secondary_y=False) # Set y-axes titles primary axis
figMSTK01.update_yaxes(title_text="Ni, Mg, Al, Mn, pH, ORP", secondary_y=True) # Set y-axes titles secondary axis

#-------------------------------------------------------------------------End----------------------------------------------------------------------------

In [ ]:
#--------------------------------------------------------------------   106VE04  ------------------------------------------------------------------------
#
#
figMSVE04 = make_subplots(specs=[[{"secondary_y": True}]])
figMSVE04_axis_truefalse_list = [True, False, False, False, True, 
                            True, True, False, False, False, 
                            False, False, False, False, True, 
                            False]
generate_plot_from_df(figMSVE04, MSVE04, figMSVE04_axis_truefalse_list)

title_plotMSVE04 = "<b>VE04_Ni Trend InterActive</b>" + '<br>' + 'Updated as of:' + str(t.strftime("%m/%d/%Y %H:%M:%S"))+'H'
figMSVE04.update_layout(title_text=title_plotMSVE04) # Adds figure title
figMSVE04.update_xaxes(title_text="DateTime") # Set x-axis title
figMSVE04.update_yaxes(title_text="<b>Predicted</b> VE04 Ni", secondary_y=False) # Set y-axes titles primary axis
figMSVE04.update_yaxes(title_text="<b>Actual</b> VE04 Ni", secondary_y=True)    # Set y-axes titles secondary axis
#
#-------------------------------------------------------------------------End----------------------------------------------------------------------------

In [ ]:
#---------   Pidata and Ni recovery calc
#
#
df_Pi_MSoprtn = pd.read_excel(r'C:\Users\v.t.flores\Documents\MS Flow 2.xlsx', sheet_name='Sheet1', index_col=False)
#Drop the row[1]
df_Pi_MSoprtn = df_Pi_MSoprtn.drop(df_Pi_MSoprtn.index[0])
#Step2: Parse the DATETIME column
df_Pi_MSoprtn['DATETIME'] = df_Pi_MSoprtn['DATETIME'].apply(lambda x: parse(str(x)))
df_Pi_MSoprtn.set_index('DATETIME', inplace=True)
df_Pi_MSoprtn
for i in df_Pi_MSoprtn.columns:
    df_Pi_MSoprtn[i] = pd.to_numeric(df_Pi_MSoprtn[i], errors='coerce')
df_Pi_MSoprtn['MS_Flow'] = np.where(df_Pi_MSoprtn['MS_FLOW_A']>df_Pi_MSoprtn['MS_FLOW_B'], df_Pi_MSoprtn['MS_FLOW_A'], df_Pi_MSoprtn['MS_FLOW_B'])
df_Pi_MSoprtn['Operation_Load'] = df_Pi_MSoprtn['MS_Flow']*100/1600
df_Pi_MSoprtn = df_Pi_MSoprtn[['MS_Flow', 'Operation_Load']]
dfn = df_Pi_MSoprtn['Operation_Load']
Ni = MSTK01[['106TK01_Ni','106TK01_Fe', '106TK01_pH']].join(MSVE04['106VE04_Ni_soln'], how='outer')
df_Pi_MSoprtn_Ni = Ni.join(dfn, how='outer')
#df_5 = df_4.interpolate(method='linear', limit_direction='forward', axis=0)
df_Pi_MSoprtn_Ni = df_Pi_MSoprtn_Ni.dropna(how='any')
df_Pi_MSoprtn_Ni['Ni_extraction'] = (df_Pi_MSoprtn_Ni['106TK01_Ni']-df_Pi_MSoprtn_Ni['106VE04_Ni_soln'])*100/df_Pi_MSoprtn_Ni['106TK01_Ni']
df_Pi_MSoprtn_Ni = df_Pi_MSoprtn_Ni.reset_index()
#
#-------------------------------------------------------------------------End----------------------------------------------------------------------------
#-------------MS Operation: 106VE04, 106TK01
#
figMSoperation = px.scatter(df_Pi_MSoprtn_Ni, 
                  x="Operation_Load", 
                  y="Ni_extraction", 
                  size="106TK01_Fe", 
                  color="106TK01_pH", 
                  hover_name="DATETIME", 
                  size_max=31,
                  range_color=(2.5,3.4),
                  range_y= (96, 100),
                  range_x=(30,105),
                  opacity=0.5)
title_plotMSoperation = "<b>Ni Recovery v OP Load at varying PLS pH</b>" + '<br>' + 'Size of circles = Fe' + ' ; ' + 'On Hover = 106TK01_pH'
figMSoperation.update_layout(title_text=title_plotMSoperation)
#
#-------------------------------------------------------------------------End----------------------------------------------------------------------------

In [ ]:
#-----------------------------------------------------------------Exporting to an HTML file--------------------------------------------------------------

with open(r'C:\Users\v.t.flores\Documents\Laboratory Analysis InterActive Plots 2020.html', 'w') as f:
    f.write(fig5.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(figNTRL.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(figDeZnTK03.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(figDeZnTK05.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(figMSTK01.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(figMSVE04.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(figMSoperation.to_html(full_html=False, include_plotlyjs='cdn'))

#-------------------------------------------------------------------------End----------------------------------------------------------------------------